In [ ]:
import pandas as pd
import seaborn as sns
from scipy import stats
import pylab as pl
import numpy as np
import arviz as az
import pickle
%matplotlib inline

In [ ]:
import importlib
import support
importlib.reload(support)
from support import CachedStanModel, get_metric, forest_plot, load_rates

In [ ]:
sr = load_rates()['sr']

In [ ]:
pl.figure()
sr.eval('count/time').groupby('pupil').hist(bins=50, density=True)
pl.xlabel('rate (sp/sec)')
pl.ylabel('PDF')

In [ ]:
sr_model = CachedStanModel('sr_fit.stan')

In [ ]:
sr_unstacked = sr.unstack('pupil')
all_index = sr_unstacked.index
data = {
    'n_cells': len(sr_unstacked),
    'spike_count_A': sr_unstacked['count'][0].astype('i').values,
    'sample_time_A': sr_unstacked['time'][0].values,
    'spike_count_B': sr_unstacked['count'][1].astype('i').values,
    'sample_time_B': sr_unstacked['time'][1].values,
}
sr_all_fit = sr_model.sampling(data, n_jobs=8, iter=20000)

In [ ]:
az.plot_trace(sr_all_fit, ['rate_A', 'rate_B', 'rate_change'])
all_summary = az.summary(sr_all_fit, credible_interval=0.9)

In [ ]:
f, axes = pl.subplots(1, 1, figsize=(4, 3))
rate_cell_change = get_metric(all_summary, 'rate_cell_change', all_index)
rate_change = get_metric(all_summary, 'rate_change')
forest_plot(axes, rate_cell_change, rate_change, 'SR (sp/sec)')